In [ ]:
# Importing all libraries
import torch
from transformers import BertTokenizer as BT,BertForMaskedLM as BFMLM
from torch.utils.data import Dataset as Ds, DataLoader as Dl
import pandas as pd

In [ ]:
train=pd.read_csv("/content/drive/MyDrive/MarianCG-NL-to-Code-master/Data/CoNaLa/train.csv")
validate=pd.read_csv("/content/drive/MyDrive/MarianCG-NL-to-Code-master/Data/CoNaLa/valid.csv")
test=pd.read_csv("/content/drive/MyDrive/MarianCG-NL-to-Code-master/Data/CoNaLa/test.csv")
# train 11125 X 2
# validate 1237 X 2
test # 500 X 2

,intent,snippet
0,send a signal `signal.sigusr1` to the current ...,"os.kill(os.getpid(), signal.SIGUSR1)"
1,decode a hex string '4a4b4c' to utf-8.,bytes.fromhex('4a4b4c').decode('utf-8')
2,check if all elements in list `mylist` are ide...,all(x == myList[0] for x in myList)
3,format number of spaces between strings `pytho...,"print('%*s : %*s' % (20, 'Python', 20, 'Very G..."
4,how to convert a string from cp-1251 to utf-8?,d.decode('cp1251').encode('utf8')
...,...,...
495,match urls whose domain doesn't start with `t`...,"re.findall('http://[^t][^s""]+\\.html', document)"
496,split a string `mystring` considering the spac...,"mystring.replace(' ', '! !').split('!')"
497,open file `path` with mode 'r',"open(path, 'r')"
498,sum elements at the same index in list `data`,[[sum(item) for item in zip(*items)] for items...


In [ ]:
from datasets import Dataset

train_dataset=Dataset.from_pandas(train)
validation_dataset=Dataset.from_pandas(validate)
test_dataset=Dataset.from_pandas(test)

# train_dataset
# validation_dataset
test_dataset

Dataset({
    features: ['intent', 'snippet'],
    num_rows: 500
})

In [ ]:
# Initialising Tokenizer and Model
BERT_tokenizer=BT.from_pretrained('bert-base-uncased')
BERT_model=BFMLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Tokenization Function
def myTokenize(data):
  return BERT_tokenizer(data['intent'],data['snippet'],truncation=True,padding='max_length')

tokenized_trained_data=train_dataset.map(myTokenize,batched=True)
tokenized_validation_data=validation_dataset.map(myTokenize,batched=True)
tokenized_test_data=test_dataset.map(myTokenize,batched=True)

tokenized_trained_data

Map:   0%|          | 0/11125 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['intent', 'snippet', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 11125
})

In [ ]:
# Training Parameters
batchSize,epochs,learningRate=1,1,5e-3

# Define Data Loader
tokenized_train_data_loader=Dl(tokenized_trained_data,batch_size=batchSize,shuffle=True)
tokenized_validation_data_loader=Dl(tokenized_validation_data,batch_size=batchSize,shuffle=True)
tokenized_test_data_loader=Dl(tokenized_test_data,batch_size=batchSize,shuffle=True)

# tokenized_train_data_loader
# tokenized_validation_data_loader
tokenized_test_data_loader

In [ ]:
# defining optimiser
BERT_optimiser=torch.optim.AdamW(BERT_model.parameters(),lr=learningRate)

# define loss function
BERT_loss_function=torch.nn.CrossEntropyLoss()

In [ ]:
# Creating device so that we can move tensors to the GPU if available
import torch

# Check if GPU is available
if torch.cuda.is_available():
  device=torch.device('cuda')
else:
  device=torch.device('cpu')

# Create a tensor
tensor=torch.randn(3,3)

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Step 1: Load your dataset
data = pd.read_csv('data.csv')
# Assuming your dataset contains columns 'text' and 'label'
train_texts = data['intent'].tolist()
train_labels = data['snippet'].tolist()

# Step 2: Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# Step 3: Define Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Step 4: Training Arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=3,               # total number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=64,    # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',             # directory for storing logs
)

# Step 5: Trainer
trainer = Trainer(
    model=model,                      # the instantiated Transformers model to be trained
    args=training_args,               # training arguments, defined above
    train_dataset=train_dataset,      # training dataset
)

# Step 6: Train the Model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Load your dataset
data = pd.read_csv('data.csv')
train_texts = data['intent'].tolist()
train_labels = data['snippet'].tolist()

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# Prepare Dataset and DataLoader
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    # Convert the label to numerical format
    if self.labels[idx] is str:
      label = int(self.labels[idx])  # Assuming your labels are in string format
      item['labels'] = torch.tensor(label)
    return item

  def __len__(self):
    return len(self.labels)
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Initialize Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training Loop
model.train()
for epoch in range(2):
  for batch in train_loader:
    optimizer.zero_grad()
    inputs = {key: batch[key].to(device) for key in batch if key != 'snippets'}  # Assuming 'intents' are the inputs
    labels = batch['snippets'].to(device)  # Assuming labels are provided in a tensor format
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyError: 'snippets'

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming you have loaded your dataset into train_df and test_df
# train_df and test_df should contain 'intent' and 'snippet' columns

# Step 1: Load and preprocess your dataset
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Step 2: Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(train_df['snippet'].unique()))

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
train_df['snippet_encoded'] = label_encoder.fit_transform(train_df['snippet'])

# Now 'snippet_encoded' column contains numerical representations of the labels
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['intent']
        label = self.data.iloc[idx]['snippet_encoded']  # Use encoded label

        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        encoding['labels'] = torch.tensor(label)  # Assuming label is already encoded
        return encoding

# Step 4: Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Step 5: Create instances of the custom dataset
train_dataset = CustomDataset(train_data, tokenizer, max_length=64)
val_dataset = CustomDataset(val_data, tokenizer, max_length=64)

# Step 6: Define data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Step 7: Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Step 8: Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

for epoch in range(3):  # Adjust number of epochs as needed
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Step 9: Evaluation loop
model.eval()
val_loss = 0
correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Loss: {val_loss / len(val_loader)}, Accuracy: {(correct / total) * 100}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: too many values to unpack (expected 2)